In [ ]:
pip install langchain_core langchain_groq mcp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.8/72.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.7 MB/s eta 0:00:00


In [ ]:
import os
import getpass
import requests
import json
import asyncio
from langchain_groq import ChatGroq
from langchain.memory import ConversationSummaryMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import ConversationChain
from mcp.client.stdio import stdio_client
from mcp import ClientSession, StdioServerParameters

# Set up API keys
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] =""
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = ""
if "GOOGLE_CSE_ID" not in os.environ:
    os.environ["GOOGLE_CSE_ID"] =""
# Initialize the ChatGroq model
llm = ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5,
    max_tokens=1000,
    timeout=None,
    max_retries=2
)

# Set up memory to store summarized conversation history
memory = ConversationSummaryMemory(
    llm=llm,
    return_messages=True
)

# Define the prompt template with legal assistant tone and PSE integration
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a highly qualified legal assistant with expertise in providing accurate and precise legal information. Respond to all inquiries using formal legal terminology, maintaining a professional and objective tone. Provide clear, concise, and well-reasoned answers, citing relevant legal principles or concepts where applicable. When relevant, incorporate real-time search results from the Google Programmable Search Engine (accessed via MCP) to provide up-to-date legal information, citing sources where applicable. If a question falls outside your knowledge or requires specific jurisdictional details, advise the user to consult a licensed attorney. Use the summarized conversation history to ensure continuity and context in responses."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{user_input}")
])

# Create a conversation chain with memory
chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    input_key="user_input",
    verbose=False  # Disable verbose logging to suppress summary output
)






In [ ]:


# MCP client for Google Programmable Search Engine (simulated)
class GooglePSEClient:
    def __init__(self):
        self.server_params = StdioServerParameters(
            command="npx",
            args=["-y", "@modelcontextprotocol/server-google-cse"],
            env={
                "GOOGLE_API_KEY": os.environ["GOOGLE_API_KEY"],
                "GOOGLE_CSE_ID": os.environ["GOOGLE_CSE_ID"]
            }
        )
        self.client = None

    async def initialize(self):
        try:
            self.client = stdio_client(self.server_params)
            await self.client.initialize()
            print("MCP client initialized for Google Programmable Search Engine.")
        except Exception as e:
            print(f"Error initializing MCP client: {e}")

    async def search(self, query):
        try:
            response = await self.client.call_tool("search_google_cse", {"query": query})
            return response.get("results", [])
        except Exception:
            # Fallback to direct Custom Search JSON API call
            url = "https://www.googleapis.com/customsearch/v1"
            params = {
                "key": os.environ["GOOGLE_API_KEY"],
                "cx": os.environ["GOOGLE_CSE_ID"],
                "q": query
            }
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response.json().get("items", [])
            return []

    async def close(self):
        if self.client:
            await self.client.close()

# Interactive conversation loop with MCP integration
async def chat_loop():
    pse_client = GooglePSEClient()
    await pse_client.initialize()

    print("Legal Assistant activated with Google Programmable Search Engine integration. Please provide your legal query (type 'exit' to quit):")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            await pse_client.close()
            break

        # Check if the query requires a search
        if any(keyword in user_input.lower() for keyword in ["recent", "current", "latest"]):
            search_results = await pse_client.search(user_input)
            context = f"Search results: {json.dumps(search_results[:2], indent=2)}"
            augmented_input = f"{user_input}\n\nContext from Google Programmable Search Engine: {context}"
        else:
            augmented_input = user_input

        response = chain.invoke({"user_input": augmented_input})
        print("Legal Assistant:", response['response'])

Error initializing MCP client: '_AsyncGeneratorContextManager' object has no attribute 'initialize'
Legal Assistant activated with Google Programmable Search Engine integration. Please provide your legal query (type 'exit' to quit):
You: exit


AttributeError: '_AsyncGeneratorContextManager' object has no attribute 'close'